# Collision Avoidance - Train Model

Welcome to this host side Jupyter Notebook!  This should look familiar if you ran through the notebooks that run on the robot.  In this notebook we'll train our image classifier to detect two classes
``free`` and ``blocked``, which we'll use for avoiding collisions.  For this, we'll use a popular deep learning library *PyTorch*

In [29]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [30]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/tello/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/tello


### Create dataset instance

Now we use the ``ImageFolder`` dataset class available with the ``torchvision.datasets`` package.  We attach transforms from the ``torchvision.transforms`` package to prepare the data for training.  

In [31]:
dataset = datasets.ImageFolder(
    'collision_avoidance',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

### Split dataset into train and test sets

Next, we split the dataset into *training* and *test* sets.  The test set will be used to verify the accuracy of the model we train.

In [32]:
train_len = int(len(dataset)*0.5)
valid_len = len(dataset) - train_len
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_len, valid_len])

In [33]:
print(len(train_dataset))
print(len(valid_dataset))

779
779


### Create data loaders to load data in batches

We'll create two ``DataLoader`` instances, which provide utilities for shuffling data, producing *batches* of images, and loading the samples in parallel with multiple workers.

In [34]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

### Define the neural network

Now, we define the neural network we'll be training.  The *torchvision* package provides a collection of pre-trained models that we can use.

In a process called *transfer learning*, we can repurpose a pre-trained model (trained on millions of images) for a new task that has possibly much less data available.

Important features that were learned in the original training of the pre-trained model are re-usable for the new task.  We'll use the ``alexnet`` model.

In [35]:
model = models.alexnet(pretrained=True)

The ``alexnet`` model was originally trained for a dataset that had 1000 class labels, but our dataset only has two class labels!  We'll replace
the final layer with a new, untrained layer that has only two outputs.  

In [36]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

Finally, we transfer our model for execution on the GPU

In [37]:
device = torch.device('cuda')
model = model.to(device)

### Train the neural network

Using the code below we will train the neural network for 30 epochs, saving the best performing model after each epoch.

> An epoch is a full run through our data.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
cont = 0
for images, labels in iter(train_loader):
    if cont >= 50:
        break
    cont += 1
    img = np.reshape(images[0], (224, 224, 3))
    plt.imshow(img)
    plt.show()
    print(labels[0])

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_f1 = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    valid_fp_count = 0.0
    valid_fn_count = 0.0
    valid_tp_count = 0.0
    valid_tn_count = 0.0
    valid_accuracy = 0.0
    for images, labels in iter(valid_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        if len(labels[labels==0]) > 0:
            valid_fn_count += float(torch.sum(labels[labels==0] != outputs[labels==0].argmax(1)))
            valid_tp_count += float(torch.sum(labels[labels==0] == outputs[labels==0].argmax(1)))
        if len(labels[labels==1]) > 0:
            valid_fp_count += float(torch.sum(labels[labels==1] != outputs[labels==1].argmax(1)))
            valid_tn_count += float(torch.sum(labels[labels==1] == outputs[labels==1].argmax(1)))
        valid_accuracy += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    precision = valid_tp_count/(valid_tp_count + valid_fp_count)
    recall = valid_tp_count/(valid_tp_count + valid_fn_count)
    valid_accuracy /= len(valid_loader)
    
    print("Precision ", precision)
    print("Recall ", recall)
    print("fp ", valid_fp_count)
    print("fn ", valid_fn_count)
    print("tp ", valid_tp_count)
    print("tn ", valid_tn_count)
    print("Accuracy ", valid_accuracy)
    
    valid_f1 = 2*precision*recall/(precision+recall)
    print('%d: %f' % (epoch, valid_f1))
    if valid_f1 > best_f1:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_f1 = valid_f1

Precision  0.874251497005988
Recall  0.44242424242424244
fp  21.0
fn  184.0
tp  146.0
tn  428.0
Accuracy  15.76923076923077
0: 0.587525
Precision  0.9074074074074074
Recall  0.593939393939394
fp  20.0
fn  134.0
tp  196.0
tn  429.0
Accuracy  11.846153846153847
1: 0.717949
Precision  0.8719723183391004
Recall  0.7636363636363637
fp  37.0
fn  78.0
tp  252.0
tn  412.0
Accuracy  8.846153846153847
2: 0.814216
Precision  0.8544303797468354
Recall  0.8181818181818182
fp  46.0
fn  60.0
tp  270.0
tn  403.0
Accuracy  8.153846153846153
3: 0.835913
Precision  0.8033707865168539
Recall  0.8666666666666667
fp  70.0
fn  44.0
tp  286.0
tn  379.0
Accuracy  8.76923076923077
4: 0.833819
Precision  0.8258426966292135
Recall  0.8909090909090909
fp  62.0
fn  36.0
tp  294.0
tn  387.0
Accuracy  7.538461538461538
5: 0.857143
Precision  0.9154929577464789
Recall  0.7878787878787878
fp  24.0
fn  70.0
tp  260.0
tn  425.0
Accuracy  7.230769230769231
6: 0.846906


Once that is finished, you should see a file ``best_model.pth`` in the Jupyter Lab file browser.  Select ``Right click`` -> ``Download`` to download the model to your workstation